# This analysis and code was performed and written by Cameron Soulette (csoulette@gmail.com)

In [1]:
#Hot imports
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Breakdown
- There are several categories of smokers. I will do both the z-score and IQR analysis.
- For the IQR analysis, I will use "lifelong non-smoker" as the "normal"

In [2]:
# Now make reference dictionary for TCGA IDs belonging to each category.
# The juncbase table is actually indexed by "CGHub_Tumor_analysis_id_RNA"
# And the smoking status is indexed by "Smoking_History"
# I'll use these headers rather than defines column numbers.

table = "./campbell_et_al_patient_characteristics.csv"
smokingRef = dict()
otherRef = dict()
out = open("names_smoking.tsv","w")
with open(table,'r') as lines:
    header = next(lines).rstrip().split(",")
    headerDict = {colName:pos for pos,colName in enumerate(header,0)}
    smokerStatus = headerDict["Smoking_History"]
    analysisID = headerDict["CGHub_Tumor_analysis_id_RNA"]
    
    for line in lines:
        cols = line.rstrip().split(",")
        if "TCGA" not in cols[0]:
            continue
        
        status, sample = cols[smokerStatus].lower(), cols[analysisID]
        if status not in smokingRef:
            smokingRef[status] = set()
        print(status,sample, sep="\t", file=out)
        smokingRef[status].add(sample)
        otherRef[sample] = status
out.close()

In [3]:
table = "./campbell_et_al_patient_characteristics.csv"
tcgaRef = dict()
with open(table,'r') as lines:
    header = next(lines).rstrip().split(",")
    headerDict = {colName:pos for pos,colName in enumerate(header,0)}
    smokerStatus = headerDict["Smoking_History"]
    analysisID = headerDict["CGHub_Tumor_analysis_id_RNA"]
    
    for line in lines:
        cols = line.rstrip().split(",")
        tcgaRef[cols[0]] = cols[analysisID]

u2muts = dict()
table = "./molecular_summary.csv"
with open(table,'r') as lines:
    header = next(lines).rstrip().split(",")
    headerDict = {colName:pos for pos,colName in enumerate(header,0)}
    for num, line in enumerate(lines,0):
        cols = line.rstrip().split(",")
        
        tID = cols[0]
        
        s34f = cols[headerDict["U2AF1_Mutations"]]
        if "S34F" in s34f:
            u2muts[tID] = True
            print(tcgaRef[tID], tID)

NA LUAD-E01317
NA LUAD-F00282
NA LUAD-S01302
NA LUAD-S01381
d1ec7dfb-965d-4679-8298-4c70bba54c03 LUAD-TCGA-49-4505
f35e0cc0-27d3-4c01-86ca-f5e9297969c7 LUAD-TCGA-49-6744
f878725e-f9dd-41c7-83e8-83344baec474 LUAD-TCGA-50-5941
676bf140-ab94-40fd-96c7-47353b700331 LUAD-TCGA-50-8460
18834dd7-861d-45b6-9c1a-079f645c8f0a LUAD-TCGA-55-1595
b1db98e2-4578-4680-a09f-c63ae45e83bd LUAD-TCGA-55-7727
0f39ade9-ed29-48c1-9939-f2bd1b4eda12 LUAD-TCGA-55-7903
c9537649-9c8f-4963-be87-34ddbd438c03 LUAD-TCGA-64-1680
775b760b-2421-4237-9f74-df239090fcc1 LUAD-TCGA-78-7145
0d4b3ea4-715b-483e-af1d-9540444eb178 LUAD-TCGA-78-8655
0718f366-4b85-44a9-9e6e-b2c5bb7dac2d LUAD-TCGA-MP-A4T4


In [4]:
normalRef = set()
with open("../gdc_legacy_LUAD_manifest_20180823.tsv",'r') as lines:
    for line in lines:
        cols = line.rstrip().split("\t")
        if "normal" in cols[1].lower():
            normalRef.add(cols[-1])

In [6]:
# Now to get the juncbase table
jbTable = "../LUAD_601_JBrunFINAL/LUAD_601_RNA_JBoutput_AS_exclusion_inclusion_counts_lenNorm.txt"


smokingColRef = dict()
normalKey = "lifelong non-smoker"
readSupportThreshold = 25
nanThreshold = .5
x, y = list(), list()
out = open("test3.tsv","w")

sampleCounts = dict()
with open(jbTable,'r') as lines:
    header = next(lines).rstrip().split("\t")
    headerDict = {colName:pos for pos,colName in enumerate(header,0)}

    # Compile column positions for each group of smokers
    # lifelong non-smoker will be my "normal" set
    allPats = set()
    normIndices = [headerDict[x] for x in normalRef]
    for status, samples in smokingRef.items():

        colNums = [headerDict[sample] for sample in list(samples) if sample in headerDict]
        smokingColRef[status] = colNums
        allPats = allPats.union(colNums)
        
    allPats = list(allPats)
    sampleCounts = {header[pos]:0 for pos in allPats}
    for num, line in enumerate(lines,0):
        

        cols = line.rstrip().split("\t")
        
        if "jcn_only" in cols[1]:
            continue
        elif "intron" in cols[1] and cols[0] == "N":
            continue
        
        normVals = [list(map(int, cols[pos].split(";"))) for pos in normIndices]

        normVals = np.asarray([x[0]/(x[0]+x[1]) if (x[0]+x[1])>readSupportThreshold else np.nan 
                              for x in normVals], 
                               dtype=float)
        
        
        nanCount = np.count_nonzero(np.isnan(normVals))

        if (nanCount/len(normVals)) > nanThreshold:
            continue
            
        
        
        median = np.nanmedian(normVals)
        firstQ = np.nanpercentile(normVals, 25)
        thirdQ = np.nanpercentile(normVals, 75)
        iqrT = (thirdQ - firstQ)*1.5
        
        allVals = [list(map(int, cols[pos].split(";"))) for pos in allPats]

        allVals = np.asarray([x[0]/(x[0]+x[1]) if (x[0]+x[1])>readSupportThreshold else np.nan 
                              for x in allVals], 
                               dtype=float)
        
        for num2,pos in enumerate(allPats,0):
            val = allVals[num2]
            sample = header[pos]
            
            if abs((val-median)) > 0.1 and abs((val-median)) > iqrT:
                sampleCounts[sample] += 1
            else:
                continue
       
for k,v in sampleCounts.items():
    print(k,v,otherRef[k],  sep="\t", file=out)
out.close()